# En France, le député assidu présente-t-il un profil type ? 

Alors que la société française se caractérise par une grande défiance de la population vis-à-vis de ses élus et responsables politiques, nous nous pencherons dans ce projet sur la question de l'assiduité des députés au cours de leur mandat. Nous verrons notamment grâce à différents outils (boîtes de Tukey, matrice de nuages de points, analyse en composantes principales, clustering, régression linéaire...) si l'on peut dresser un portrait type pour caractériser le député assidu ou absentéiste. 

In [1]:
%matplotlib widget

import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as plt3d
import seaborn as sns

## I. Récupération et préparation des données

### 1. Importation des données

#### 1.1 Récupération de la base de données principale

On récupère sur le site citoyen nosdeputes.fr une base de données synthétisant l'activité parlementaire sur les 12 derniers mois. Pour chaque député, cette table, que nous avons convertie au format .tsv, contient des informations relatives à son état civil, à la circonscription dont il est le représentant, à son éventuel parti politique et surtout à son travail parlementaire (semaines de présence à l'Assemblée nationale, rédaction de rapports, participation à des commissions...).

In [2]:
# Conversion du fichier tsv en DataFrame.
df = pd.read_csv("../Données/nosdeputes.fr_synthese_2020-10-24.tsv", sep='\t')
df

,id,nom,nom_de_famille,prenom,sexe,date_naissance,lieu_naissance,num_deptmt,nom_circo,num_circo,...,hemicycle_interventions,hemicycle_interventions_courtes,amendements_proposes,amendements_signes,amendements_adoptes,rapports,propositions_ecrites,propositions_signees,questions_ecrites,questions_orales
0,32,Damien Abad,Abad,Damien,H,1980-04-05,Nîmes (Gard),01,Ain,5,...,253,307,19,1736,63,0,0,25,38,9
1,43,Caroline Abadie,Abadie,Caroline,F,1976-09-07,Saint-Martin-d'Hères (Isère),38,Isère,8,...,17,8,14,362,196,0,0,3,3,0
2,493,Jean-Félix Acquaviva,Acquaviva,Jean-Félix,H,1973-03-19,Bastia (Haute-Corse),2B,Haute-Corse,2,...,109,9,186,2309,89,0,0,4,22,3
3,152,Lénaïck Adam,Adam,Lénaïck,H,1992-02-19,Saint Laurent du Maroni (Guyane),973,Guyane,2,...,3,0,15,581,175,0,0,4,1,0
4,234,Damien Adam,Adam,Damien,H,1989-06-28,Orléans (Loiret),76,Seine-Maritime,1,...,24,7,74,687,225,0,0,4,10,2
5,135,Saïd Ahamada,Ahamada,Saïd,H,1972-11-07,Saint-Denis (Réunion),13,Bouches-du-Rhône,7,...,8,1,14,394,200,0,0,4,0,1
6,508,Éric Alauzet,Alauzet,Éric,H,1958-06-07,Nancy (Meurthe-et-Moselle),25,Doubs,2,...,3,1,14,541,191,0,0,4,19,0
7,183,Ramlati Ali,Ali,Ramlati,F,1961-05-28,Pamandzi (Mayotte),976,Mayotte,1,...,5,2,18,485,163,0,0,3,2,4
8,499,Aude Amadou,Amadou,Aude,F,1980-02-29,Coutances (Manche),44,Loire-Atlantique,4,...,2,1,8,334,144,0,0,3,0,0
9,382,Patrice Anato,Anato,Patrice,H,1976-03-14,Lomé (Togo),93,Seine-Saint-Denis,3,...,5,0,16,574,197,1,0,4,16,2


#### 1.2 Complétion de la base de données à l'aide de webscraping

Ce jeu de données est plutôt complet, mais il lui manque une variable qui pourrait nous intéresser pour notre étude : le statut du député (sortant, élu pour la première fois, ancien député ou arrivé en cours de mandat).

Nous allons donc compléter la table avec la variable "statut" du tableau disponible sur la page http://www2.assemblee-nationale.fr/elections/liste/2017/resultats/RESULTAT.

In [3]:
# On récupère le code source de la page afin d'en extraire le tableau.
from urllib import request
import bs4

request_text = request.urlopen("http://www2.assemblee-nationale.fr/elections/liste/2017/resultats/RESULTAT").read()
page = bs4.BeautifulSoup(request_text, "html")
tableau_html = page.find("table") # On extrait le tableau d'intérêt de la page HTML.

Nous avons récupéré le code HTML du tableau, récupérons maintenant ses entêtes.

In [4]:
entetes = tableau_html.find('thead')
entetes = entetes.find('tr')
entetes = entetes.find_all('th')
entetes = [entete.text.strip() for entete in entetes]
print(entetes)

['Civ.', 'Nom', 'Prénom', 'Département', 'Circ.', 'Statut', 'Tour', 'Nuance']


Puis complétons un dictionnaire avec ses lignes et transformons le en DataFrame.

In [5]:
dict_tableau = {}
for entete in entetes:
    dict_tableau[entete] = []

corps_tableau = tableau_html.find('tbody')
lignes_tableau = corps_tableau.find_all('tr')
for ligne in lignes_tableau:
    colonnes = ligne.find_all('td')
    for i, element in enumerate(colonnes):
        dict_tableau[entetes[i]].append(element.text.strip())

df1 = pd.DataFrame.from_dict(dict_tableau)

On affiche les premières lignes pour s'assurer qu'on obtient bien le tableau souhaité.

In [6]:
df1.head()

,Civ.,Nom,Prénom,Département,Circ.,Statut,Tour,Nuance
0,M.,ABAD,DAMIEN,AIN,5,SORTANT,2,LR
1,Mme,ABADIE,CAROLINE,ISERE,8,ELUE POUR LA 1ERE FOIS,2,REM
2,Mme,ABBA,BÉRANGÈRE,HAUTE-MARNE,1,ELUE POUR LA 1ERE FOIS,2,REM
3,M.,ACQUAVIVA,JEAN-FÉLIX,HAUTE-CORSE,2,ELU POUR LA 1ERE FOIS,2,REG
4,M.,ADAM,LÉNAÏCK,GUYANE,2,ELU POUR LA 1ERE FOIS,2,REM


### 2. Nettoyage des données

Maintenant que nous disposons de nos deux bases, nous allons travailler sur celles-ci de sorte à les rendre plus maniables : nous procédons donc au nettoyage des données.

Dans le tableau webscrapé, conservons uniquement les variables "Nom", "Prénom" et "Statut" et convertissons la casse des modalités et des variables, dans l'optique de les comparer avec celles de la table principale. 

In [7]:
df1 = df1[['Nom','Prénom','Statut']] # On ne conserve que 3 variables.
# On modifie la casse des modalités.
df1["Nom"] = df1["Nom"].str.lower()
df1["Prénom"] = df1["Prénom"].str.lower()
df1["Statut"] = df1["Statut"].str.lower()
df1

,Nom,Prénom,Statut
0,abad,damien,sortant
1,abadie,caroline,elue pour la 1ere fois
2,abba,bérangère,elue pour la 1ere fois
3,acquaviva,jean-félix,elu pour la 1ere fois
4,adam,lénaïck,elu pour la 1ere fois
5,adam,damien,elu pour la 1ere fois
6,ahamada,saïd,elu pour la 1ere fois
7,alauzet,éric,sortant
8,ali,ramlati,elue pour la 1ere fois
9,aliot,louis,elu pour la 1ere fois


On concatène le nom et le prénom pour ne conserver que le patronyme complet et le comparer avec celui de l'autre base. 

In [8]:
df1['Nom'] = df1['Prénom'] + ' ' + df1['Nom'] 
df1 = df1.drop(['Prénom'], axis=1) # On retire donc la variable 'Prénom', désormais inutile.

Afin d'uniformiser les styles d'écriture, on enlève désormais les mots vides (ici les déterminants "de" fréquemment présents dans les noms de famille), car ils ne sont pas écrits de la même manière dans les deux bases. 

In [9]:
replace_values_ean = {' de ':' ', ' (de) ':' '}

# On crée la fonction de nettoyage qui retire le déterminant "de".
def clean_dataset(data): 
    data.replace({'nom': replace_values_ean, 'Nom' : replace_values_ean},regex=True,inplace=True)
    return data

In [10]:
# On crée une copie de la table principale pour ne pas modifier la base initiale.
df_new = df.copy() 
df_new['nom'] = df_new['nom'].str.lower()
clean_dataset(df_new) # On procède au nettoyage sur la base initiale. 

,id,nom,nom_de_famille,prenom,sexe,date_naissance,lieu_naissance,num_deptmt,nom_circo,num_circo,...,hemicycle_interventions,hemicycle_interventions_courtes,amendements_proposes,amendements_signes,amendements_adoptes,rapports,propositions_ecrites,propositions_signees,questions_ecrites,questions_orales
0,32,damien abad,Abad,Damien,H,1980-04-05,Nîmes (Gard),01,Ain,5,...,253,307,19,1736,63,0,0,25,38,9
1,43,caroline abadie,Abadie,Caroline,F,1976-09-07,Saint-Martin-d'Hères (Isère),38,Isère,8,...,17,8,14,362,196,0,0,3,3,0
2,493,jean-félix acquaviva,Acquaviva,Jean-Félix,H,1973-03-19,Bastia (Haute-Corse),2B,Haute-Corse,2,...,109,9,186,2309,89,0,0,4,22,3
3,152,lénaïck adam,Adam,Lénaïck,H,1992-02-19,Saint Laurent du Maroni (Guyane),973,Guyane,2,...,3,0,15,581,175,0,0,4,1,0
4,234,damien adam,Adam,Damien,H,1989-06-28,Orléans (Loiret),76,Seine-Maritime,1,...,24,7,74,687,225,0,0,4,10,2
5,135,saïd ahamada,Ahamada,Saïd,H,1972-11-07,Saint-Denis (Réunion),13,Bouches-du-Rhône,7,...,8,1,14,394,200,0,0,4,0,1
6,508,éric alauzet,Alauzet,Éric,H,1958-06-07,Nancy (Meurthe-et-Moselle),25,Doubs,2,...,3,1,14,541,191,0,0,4,19,0
7,183,ramlati ali,Ali,Ramlati,F,1961-05-28,Pamandzi (Mayotte),976,Mayotte,1,...,5,2,18,485,163,0,0,3,2,4
8,499,aude amadou,Amadou,Aude,F,1980-02-29,Coutances (Manche),44,Loire-Atlantique,4,...,2,1,8,334,144,0,0,3,0,0
9,382,patrice anato,Anato,Patrice,H,1976-03-14,Lomé (Togo),93,Seine-Saint-Denis,3,...,5,0,16,574,197,1,0,4,16,2


In [11]:
clean_dataset(df1) # On procède au nettoyage sur la table webscrapée. 

,Nom,Statut
0,damien abad,sortant
1,caroline abadie,elue pour la 1ere fois
2,bérangère abba,elue pour la 1ere fois
3,jean-félix acquaviva,elu pour la 1ere fois
4,lénaïck adam,elu pour la 1ere fois
5,damien adam,elu pour la 1ere fois
6,saïd ahamada,elu pour la 1ere fois
7,éric alauzet,sortant
8,ramlati ali,elue pour la 1ere fois
9,louis aliot,elu pour la 1ere fois


On élabore maintenant une fonction qui retire les accents.

In [12]:
import unicodedata,string
import unidecode

def remove_accent(s) :
    return ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

In [13]:
# Retrait des accents sur tous les noms complets des députés dans les deux bases.
df_new["nom"]=df_new["nom"].map(lambda x: remove_accent(x))
df1["Nom"]=df1["Nom"].map(lambda x: remove_accent(x))

On peut désormais joindre les deux tables pour ajouter à la table initiale la variable portant sur le statut du député. 

In [14]:
df_work = pd.merge(df_new, df1, how='left', left_on='nom', right_on='Nom')

In [15]:
df_work = df_work.drop('Nom', axis=1) # On retire la variable "Nom" qui fait désormais doublon.

On va remplacer les valeurs manquantes pour la variable "statut" par la modalité "arrivé en cours de mandat". En effet, cela concerne des députés présents dans la base mis à jour mais non présents dans la base webscrapée qui correspond aux députés élus en 2017 ; ils sont donc arrivés au cours des trois dernières années. 

In [16]:
for i in range(len(df_work)) :
    if df_work['Statut'].isnull()[i] == True :
        df_work['Statut'][i] = 'arrive en cours de mandat'

On crée désormais une variable âge à partir de la variable date_naissance. Cela va nous permettre de répartir les députés en tranches d'âge de façon à analyser l'influence de l'âge sur l'assiduité. 

In [17]:
import datetime

df_work['age'] = 0 # On initialise à 0 la valeur de la variable age pour chaque individu.
adj = datetime.date.today()
for i in range(len(df_work)) :
    date = datetime.datetime.strptime(df_work["date_naissance"][i], '%Y-%m-%d')
    df_work['age'][i] = adj.year - date.year - ((adj.month, adj.day) < (date.month, date.day))
    # On calcule la différence entre l'année actuelle et l'année de naissance de chaque député, en corrigeant d'une unité si l'anniversaire n'est pas encore arrivé.

df_work = df_work.sort_values("age") # On réordonne la base dans l'ordre croissant de l'âge des députés. 

# On répartit maintenant les députés dans différentes tranches d'âge. 
df_work['tranche_age'] = 0
for i in range(len(df_work)) :
    if 20 <= df_work['age'][i] < 30 :
        df_work['tranche_age'][i] = '20-30 ans'
    if 30 <= df_work['age'][i] < 40 :
        df_work['tranche_age'][i] = '30-40 ans'
    if 40 <= df_work['age'][i] < 50 :
        df_work['tranche_age'][i] = '40-50 ans'
    if 50 <= df_work['age'][i] < 60 :
        df_work['tranche_age'][i] = '50-60 ans'
    if 60 <= df_work['age'][i] < 70 :
        df_work['tranche_age'][i] = '60-70 ans'
    if df_work['age'][i] >= 70 :
        df_work['tranche_age'][i] = '+ de 70 ans'

On poursuit le nettoyage en remplaçant la modalité "0" de la variable profession par la modalité "Aucune", pour des questions de lisibilité.  

In [18]:
for i in range(len(df_work)) :
    if df_work['profession'][i] == '0' :
        df_work['profession'][i] = 'Aucune'

On peut maintenant analyser les variables présentes dans la base définitive df_work afin de sélectionner celles que nous pouvons éliminer dans le cadre de notre étude. 

In [19]:
df_work.columns # On affiche la liste des variables.

Index(['id', 'nom', 'nom_de_famille', 'prenom', 'sexe', 'date_naissance',
       'lieu_naissance', 'num_deptmt', 'nom_circo', 'num_circo',
       'mandat_debut', 'mandat_fin', 'ancien_depute', 'groupe_sigle',
       'parti_ratt_financier', 'sites_web', 'emails', 'anciens_mandats',
       'profession', 'place_en_hemicycle', 'url_an', 'id_an', 'slug',
       'url_nosdeputes', 'url_nosdeputes_api', 'nb_mandats', 'twitter',
       'semaines_presence', 'commission_presences', 'commission_interventions',
       'hemicycle_interventions', 'hemicycle_interventions_courtes',
       'amendements_proposes', 'amendements_signes', 'amendements_adoptes',
       'rapports', 'propositions_ecrites', 'propositions_signees',
       'questions_ecrites', 'questions_orales', 'Statut', 'age',
       'tranche_age'],
      dtype='object')

In [20]:
# On retire les variables que l'on juge inutiles pour notre étude.
df_work = df_work.drop(['id', 'nom_de_famille', 'prenom', 'date_naissance', 'lieu_naissance','mandat_debut', 'mandat_fin', 'ancien_depute','parti_ratt_financier', 'sites_web','emails', 'anciens_mandats', 'place_en_hemicycle', 'url_an','id_an', 'slug', 'url_nosdeputes', 'url_nosdeputes_api', 'twitter'], axis=1)

In [21]:
df_work.head() # On regarde les premières lignes de la base de travail définitive.

,nom,sexe,num_deptmt,nom_circo,num_circo,groupe_sigle,profession,nb_mandats,semaines_presence,commission_presences,...,amendements_signes,amendements_adoptes,rapports,propositions_ecrites,propositions_signees,questions_ecrites,questions_orales,Statut,age,tranche_age
383,ludovic pajot,H,62,Pas-de-Calais,10,NI,Aucune,1,23,15,...,327,1,0,0,2,54,5,elu pour la 1ere fois,26,20-30 ans
139,typhanie degois,F,73,Savoie,1,LREM,Juriste,1,21,23,...,839,207,0,0,3,36,1,elue pour la 1ere fois,27,20-30 ans
235,pierre henriet,H,85,Vendée,5,LREM,Professeur du secondaire et technique,1,31,26,...,382,188,0,0,3,9,0,elu pour la 1ere fois,28,20-30 ans
3,lenaick adam,H,973,Guyane,2,LREM,Cadre supérieur (entreprises privée),1,8,5,...,581,175,0,0,4,1,0,elu pour la 1ere fois,28,20-30 ans
375,mickael nogal,H,31,Haute-Garonne,4,LREM,Autre profession libérale,1,29,30,...,362,176,0,0,4,0,2,elu pour la 1ere fois,29,20-30 ans


## II. Analyse descriptive

On procède à une analyse descriptive des données afin d'identifier les premières grandes tendances qui guideront notre travail. Les boîtes à moustaches (ou boîtes de Tukey, dites boxplots en anglais) permettent par exemple d'étudier les statistiques liées aux semaines de présence des députés à l'Assemblée nationale, en distinguant selon le sexe, le parti politique ou encore l'âge. On utilise ici la librairie seaborn. 

In [22]:
fig, ax = plt.subplots(1, 2, figsize=(10,4))
sns.set_style("whitegrid")
sns.histplot(data=df_work, x="semaines_presence", kde=True, ax=ax[0])
ax[0].set_xlabel('Semaines de présence')
ax[0].set_ylabel('Nombre de députés')
ax[0].set_title('Semaines de présence des députés')
sns.boxplot(y="semaines_presence",data=df_work, ax=ax[1])
ax[1].set_xlabel("Boîte de Tukey des semaines de présence")
ax[1].set_ylabel('')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
fig, ax = plt.subplots(1, 2, figsize=(10,4))
sns.set_style("whitegrid")
sns.histplot(data=df_work, x="semaines_presence", hue="sexe", kde=True, ax=ax[0])
ax[0].set_xlabel('Semaines de présence')
ax[0].set_ylabel('Nombre de députés')
ax[0].set_title('Semaines de présence des députés selon le sexe')
sns.boxplot(x="sexe", y="semaines_presence", data=df_work, ax=ax[1])
ax[1].set_xlabel("Boîte de Tukey des semaines de présence selon le sexe")
ax[1].set_ylabel('')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
fig, ax = plt.subplots(2, 1, figsize=(10,8))
sns.set_style("whitegrid")
sns.kdeplot(data=df_work, x="semaines_presence", hue="groupe_sigle", ax=ax[0])
ax[0].set_title('Distributions des semaines de présence des députés selon leur parti')
ax[0].set_xlabel('Semaines de présence')
ax[0].set_ylabel('Proportion des députés')
sns.boxplot(x="groupe_sigle", y="semaines_presence", data=df_work, ax=ax[1])
ax[1].set_xlabel("Boîte de Tukey des semaines de présence selon le parti")
ax[1].set_ylabel("")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
fig, ax = plt.subplots(2, 1, figsize=(10,8))
sns.set_style("whitegrid")
sns.kdeplot(data=df_work, x="semaines_presence", hue="tranche_age", ax=ax[0])
ax[0].set_title("Distributions des semaines de présence des députés selon leur tranche d'âge")
ax[0].set_xlabel('Semaines de présence')
ax[0].set_ylabel('Proportion des députés')
sns.boxplot(x="tranche_age", y="semaines_presence", data=df_work, ax=ax[1])
ax[1].set_xlabel("Boîte de Tukey des semaines de présence selon la tranche d'âge")
ax[1].set_ylabel("")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

L'observation de ces boîtes à moustaches donne à voir quelques premières tendances :
- certains députés se démarquent de l'immense majorité des autres députés par un très fort absentéisme, comme le montrent les valeurs extrêmes du premier graphique ;
- globalement, les hommes sont légèrement plus présents à l'Assemblée nationale que les femmes ;
- il y a d'assez grandes disparités entre les différents partis ;
- les députés les plus âgés semblent être les moins assidus, tandis que les 50-60 ans sont visiblement la génération la plus assidue.

L'analyse descriptive de nos données reste cependant très insuffisante pour répondre à la problématique initiale.

## III. Modélisation

### 3.1 Liens entre les différentes variables d'assiduité

Nous avons jusqu'à présent étudié uniquement le nombre de semaines de présence des députés sur les bancs de l'Assemblée, sans nous préoccuper des autres indicateurs d'assiduité. Nous allons nous demander dans quelle mesure la présence est suffisante pour révéler l'assiduité, notamment en étudiant les corrélations entre les différents indicateurs.

Les indicateurs d'assiduité que nous avons à disposition sont 'semaines_presence', 'commission_presences', 'commission_interventions', 'hemicycle_interventions','hemicycle_interventions_courtes', 'amendements_proposes', 'amendements_signes', 'amendements_adoptes', 'rapports', 'propositions_ecrites', 'propositions_signees','questions_ecrites' et 'questions_orales'.

In [26]:
# On trace la matrice des nuages de points afin de percevoir les premiers liens entre les variables d'assiduité.
sns.set(font_scale=0.3)
sns.pairplot(df_work[['semaines_presence', 'commission_presences', 'commission_interventions', 'hemicycle_interventions','hemicycle_interventions_courtes', 'amendements_proposes', 'amendements_signes', 'amendements_adoptes', 'rapports', 'propositions_ecrites', 'propositions_signees','questions_ecrites', 'questions_orales']], height=0.6, markers=".", aspect=1.1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

On constate de nombreuses corrélations entre les variables, cependant la variable donnant les semaines de présence semble par exemple assez décorrélée avec le nombre de questions écrites. Il faut donc prendre en compte un nombre plus élevé de variables pour dégager correctement l'assiduité des députés.

Nous allons commencer par regrouper les députés en groupes relativement homogènes (clusters) selon l'ensemble des variables d'assiduité.

### 3.2 Normalisation des données

Afin d'éviter que des variables l'emportent sur d'autres par des effets d'échelle, nous allons normaliser (centrer et réduire) les variables sur lesquelles nous obtiendrons les clusters.


In [27]:
# On crée une nouvelle table avec le numéro du député en index et les variables d'assiduité.
df_clusters = df_work[['semaines_presence', 'commission_presences', 'commission_interventions', 'hemicycle_interventions','hemicycle_interventions_courtes', 'amendements_proposes', 'amendements_signes', 'amendements_adoptes', 'rapports', 'propositions_ecrites', 'propositions_signees','questions_ecrites', 'questions_orales']]

In [28]:
import sklearn # On importe scikit-learn.
# On centre et réduit les variables de manière à effectuer une ACP normée.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
Z = sc.fit_transform(df_clusters)
Z

array([[-4.99136041e-01, -1.17818822e+00, -4.25038063e-01, ...,
        -6.82286754e-01,  2.36772151e+00,  6.48408303e-01],
       [-7.86259121e-01, -7.67223361e-01, -4.76110854e-01, ...,
        -5.05906629e-01,  1.23583513e+00, -6.12158245e-01],
       [ 6.49356279e-01, -6.13111538e-01, -4.42062327e-01, ...,
        -5.05906629e-01, -4.61994440e-01, -9.27299882e-01],
       ...,
       [-4.99136041e-01, -4.07629108e-01,  5.68528285e-04, ...,
         7.28754245e-01,  1.42448286e+00,  3.33266666e-01],
       [ 3.62233199e-01,  1.57447577e-01,  1.36762638e-01, ...,
         2.13979524e+00, -7.13524747e-01,  1.27869158e+00],
       [-2.93968222e+00, -1.69189430e+00, -5.78256436e-01, ...,
         2.13979524e+00,  9.84304822e-01, -9.27299882e-01]])

On vérifie que les moyennes sont nulles et les écarts-types unitaires ; c'est bien le cas : la normalisation a bien été effectuée.

In [29]:
print(np.mean(Z, axis=0))
print(np.std(Z, axis=0, ddof=0))

[-6.09695761e-17 -5.93217497e-17  1.48304374e-17 -4.61391387e-17
  3.29565276e-18 -6.59130553e-18 -2.63652221e-17  9.22782774e-17
 -6.42652289e-17  5.76739234e-17  9.88695829e-17  3.95478332e-17
 -1.31826111e-17]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


### 3.3 Clustering

Afin de déterminer des groupes de députés en fonction de leur assiduité, on va désormais procéder à un clustering, en s'appuyant sur la méthode des k-moyennes (k-means), très répandue en partitionnement des données. Cette étape va nous permettre de constituer des groupes de députés d'assiduité comparable ; nous dégagerons ensuite les caractéristiques partagées par les députés appartenant au même cluster.

In [30]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score # On importe la métrique "silhouette" qui nous servira à déterminer le nombre de clusters optimal. 

In [31]:
# On applique la méthode des k-means sur les variables d'assiduité centrées et réduites, en supposant pour l'instant par défaut qu'il existe 3 clusters.
kmeans = KMeans(n_clusters=3, random_state=1464855801) 
# random_state est la graine aléatoire specifiée pour rendre les résultats reproductibles.
kmeans.fit(Z)

# On trie les index en fonction des différents groupes de députés (ie des clusters).
idk = np.argsort(kmeans.labels_)

# Affichage des observations et leurs groupes
print(pd.DataFrame({"député" : df_clusters.index[idk], "cluster" : kmeans.labels_[idk]}))

     député  cluster
0       383        0
1       238        0
2       116        0
3       515        0
4       470        0
5       150        0
6        12        0
7       324        0
8       396        0
9        69        0
10      190        0
11      113        0
12      504        0
13      534        0
14      151        0
15      517        0
16       94        0
17       85        0
18      422        0
19      414        0
20      478        0
21      374        0
22      410        0
23       65        0
24      274        0
25      207        0
26      205        0
27      208        0
28      114        0
29      406        0
..      ...      ...
509     157        1
510     120        1
511     507        1
512     521        1
513      39        1
514     182        1
515     310        1
516      71        1
517     258        1
518     278        1
519       0        1
520     164        1
521     256        1
522     216        1
523     378        1
524      54  

On s'intéresse désormais à la répartition des députés au sein des différents clusters. 

In [32]:
nb_clusters = kmeans.labels_.tolist()
print(nb_clusters.count(0)) # Comptage du nombre de députés dans le cluster 1.
print(nb_clusters.count(1)) # Comptage du nombre de députés dans le cluster 2.
print(nb_clusters.count(2)) # Comptage du nombre de députés dans le cluster 3.

422
109
8


On observe que l'un des clusters, restreint à 8 observations, semble contenir des députés très marginalisés, dont nous analyserons plus tard les caractéristiques.
Vérifions tout d'abord le nombre optimal de clusters en s'appuyant sur la métrique "silhouette" : celle-ci évalue grâce à un coefficient (compris entre -1 et 1) propre à chaque point la différence entre la distance moyenne de ce point avec les points du même cluster et la distance moyenne de ce même point avec les points des autres clusters. Si le point est bien placé, il doit donc être plus proche en moyenne des points de son cluster ; ceci se matérialise par un coefficient de silhouette proche de 1. A l'inverse, un coefficient proche de -1 témoigne d'un mauvais partitionnement. On va donc tracer le score moyen de silhouette en fonction du nombre de clusters, et retenir le nombre de clusters correspondant au score maximal.

In [33]:
# On va faire varier le nombre de clusters dans un intervalle réaliste, disons de 2 à 10.
res = np.arange(9, dtype="double")
for k in np.arange(9):
    km = KMeans(n_clusters=k+2)
    km.fit(Z)
    res[k] = silhouette_score(Z, km.labels_)
print(res)

[0.39121398 0.38631287 0.15581256 0.16710517 0.18147923 0.18837786
 0.20244204 0.21017308 0.18948964]


In [34]:
plt.figure(figsize=(8,4))
plt.plot(np.arange(2,11,1), res)
plt.title("Coefficient de silhouette moyen en fonction du nombre de clusters", size=16)
plt.xlabel("Nombre de clusters", size=10)
plt.ylabel("Score de silhouette moyen", size=10)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

A la vue de ce graphique, le choix optimal semble se porter sur 2 clusters (3 clusters ne semble pas être un mauvais choix non plus). Interprétons ceux-ci au moyen d'une analyse en composantes principales (ACP).

### 3.4 Analyse en Composantes Principales

Les données étant centrées et réduites, nous pouvons débuter l'ACP.

In [35]:
from sklearn.decomposition import PCA
# Instanciation de l'objet PCA
acp = PCA(svd_solver='full')

On remarque que le nombre de composantes n'est pas spécifié, il correspond donc par défaut au nombre de variables, à savoir 13 (ce qu'on vérifie dans la cellule suivante). Nous stockons maintenant les coordonnées factorielles dans la variable coord grâce à la fonction fit_transform().

In [36]:
# Calcul des coordonnées factorielles
coord = acp.fit_transform(Z)
# Vérification du nombre de composantes principales
print(acp.n_components_)

13


Désormais, nous allons afficher le pourcentage de la variance expliquée par chaque axe factoriel.

In [37]:
print(acp.explained_variance_ratio_)

[0.30292818 0.16774724 0.10467894 0.07784485 0.07354728 0.06130974
 0.0578616  0.04991936 0.03846265 0.02819897 0.01826355 0.01285006
 0.00638759]


On remarque que le premier axe explique environ 30% de l'information disponible, et les trois premiers près de 57 % : les autres axes ne semblent à première vue pas complètement anecdotiques. Nous allons donc tracer l'éboulis des valeurs propres (qui correspondent aussi à la variance expliquée par chaque axe) afin d'appliquer la méthode dite du coude. 

In [38]:
plt.figure(figsize=(8,4))
plt.bar(np.arange(1,acp.n_components_+1),acp.explained_variance_ratio_*100)
plt.title("Part de la variance expliquée par chaque axe factoriel", size=16, fontweight='bold')
plt.ylabel("Variance expliquée par l'axe factoriel (en %)", size=10)
plt.xlabel("Axe factoriel", size=10)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Au vu du diagramme ci-dessus, nous allons choisir de garder les trois premières composantes factorielles (même si 4 composantes correspondraient mieux d'après la méthode du coude, nous choisissons 3 composantes pour des questions de représentation). Nous les représentons dans la figure en trois dimensions ci-dessous, avec une couleur différente pour chaque cluster trouvé à la section précédente.

In [39]:
fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(-5,8)
ax.set_ylim(-5,8)
ax.set_zlim(-5,8)

for i in range(len(coord[:,0])):
    if kmeans.labels_[i] == 0:
        ax.text(coord[i,0], coord[i,1], coord[i,2], str(i), size=6, color='r')
    elif kmeans.labels_[i] == 1:
        ax.text(coord[i,0], coord[i,1], coord[i,2], str(i), size=6, color='g')
    else:
        ax.text(coord[i,0], coord[i,1], coord[i,2], str(i), size=6, color='b')

ax.scatter(coord[:,0], coord[:,1], coord[:,2])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Nous pouvons constater que le nuage de points se compose de deux groupes : l'un quasiment aplati dans le plan (Oxy), qui est le premier plan factoriel, l'autre ayant des cotes elevées. Le cluster des 8 députés se distingue largement en constituant le second groupe ; les deux autres clusters semblent regroupés dans le plan (Oxy).

On constate néanmoins que les députés du petit cluster écrasent l'information donnée par les autres députés. Afin d'analyser plus finement les différents types de députés, nous allons retirer ces députés de notre étude après avoir interprété leur type.

Ces derniers députés se distinguant des autres par une cote élevée, interprétons les différents axes de l'ACP. On calcule pour cela les corrélations des variables avec chaque axe factoriel.

In [40]:
n = len(df_clusters)
p = len(df_clusters.columns)

eigval = (n-1)/n*acp.explained_variance_
sqrt_eigval = np.sqrt(eigval)
corvar = np.zeros((p,p))
for k in range(p):
    corvar[:,k] = acp.components_[k,:] * sqrt_eigval[k]

print(pd.DataFrame({'Variable':df_clusters.columns,'Cor_axe_1':corvar[:,0],'Cor_axe_2':corvar[:,1], 'Cor_axe_3':corvar[:,2]}))

                           Variable  Cor_axe_1  Cor_axe_2  Cor_axe_3
0                 semaines_presence   0.557067   0.600929  -0.191771
1              commission_presences   0.425776   0.662578  -0.302692
2          commission_interventions   0.496364   0.325125  -0.420058
3           hemicycle_interventions   0.648652   0.488061   0.485997
4   hemicycle_interventions_courtes   0.399350   0.453602   0.745994
5              amendements_proposes   0.695054  -0.235665  -0.117199
6                amendements_signes   0.763213  -0.375204  -0.062249
7               amendements_adoptes  -0.428367   0.497853  -0.223568
8                          rapports   0.027278   0.174714  -0.383634
9              propositions_ecrites   0.495685  -0.237651   0.004301
10             propositions_signees   0.587427  -0.415955   0.040091
11                questions_ecrites   0.497084  -0.334478   0.013451
12                 questions_orales   0.729486  -0.121882  -0.216111


Puis on représente ces corrélations sur la sphère unité de $\mathbb{R}^3$.

In [41]:
r = 1
pi = np.pi
cos = np.cos
sin = np.sin
phi, theta = np.mgrid[0.0:pi:100j, 0.0:2.0*pi:100j]
x = r*sin(phi)*cos(theta)
y = r*sin(phi)*sin(theta)
z = r*cos(phi)

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(x, y, z,  rstride=1, cstride=1, color='c', alpha=0.1, linewidth=0)

ax.add_line(plt3d.art3d.Line3D([-1,1], [0,0], [0,0]))
ax.add_artist(plt3d.art3d.Text3D(x=1, y=0, z=0, text='Axe 1', zdir='x', size=8))

ax.add_line(plt3d.art3d.Line3D([0,0], [-1,1], [0,0]))
ax.add_artist(plt3d.art3d.Text3D(x=0, y=1, z=0, text='Axe 2', zdir='y', size=8))

ax.add_line(plt3d.art3d.Line3D([0,0], [0,0], [-1,1]))
ax.add_artist(plt3d.art3d.Text3D(x=0, y=0, z=1, text='Axe 3', zdir='z', size=8))

ax.quiver(np.zeros(p), np.zeros(p), np.zeros(p), corvar[:,0], corvar[:,1], corvar[:,2], color='r')
for i in range(p):
    ax.add_artist(plt3d.art3d.Text3D(x=corvar[i,0], y=corvar[i,1], z=corvar[i,2], text=df_clusters.columns[i], zdir=(corvar[i,0], corvar[i,1], corvar[i,2]), size=6))


ax.set_xlim([-1,1])
ax.set_ylim([-1,1])
ax.set_zlim([-1,1])

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

On constate que le troisième axe différencie essentiellement les députés effectuant de nombreuses interventions (courtes ou non) dans l'hémicycle. Le groupe de députés se situant dans le groupe du haut sur le nuage de point est donc celui des députés les plus actifs en séance.

Retirons les et recommençons le clustering et l'ACP.

### 3.5 Affinage de l'analyse

In [42]:
df_clusters2 = df_clusters.drop(df_clusters.index[kmeans.labels_ == 2])
Z2 = sc.fit_transform(df_clusters2)

In [43]:
res = np.arange(9, dtype="double")
for k in np.arange(9):
    km = KMeans(n_clusters=k+2)
    km.fit(Z2)
    res[k] = silhouette_score(Z2, km.labels_)

plt.figure(figsize=(8,4))
plt.plot(np.arange(2,11,1), res)
plt.title("Coefficient de silhouette moyen en fonction du nombre de clusters", size=16)
plt.xlabel("Nombre de clusters", size=10)
plt.ylabel("Score de silhouette moyen", size=10)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Cette fois-ci, le nombre de clusters optimal est clairement 2.

In [44]:
kmeans2 = KMeans(n_clusters=2, random_state=3535864)
kmeans2.fit(Z2)
idk = np.argsort(kmeans2.labels_)
print(pd.DataFrame({"député" : df_clusters2.index[idk], "cluster" : kmeans2.labels_[idk]}))
nb_clusters2 = kmeans2.labels_.tolist()
print(nb_clusters2.count(0)) # Comptage du nombre de députés dans le cluster 1.
print(nb_clusters2.count(1)) # Comptage du nombre de députés dans le cluster 2.

     député  cluster
0       383        0
1       114        0
2       208        0
3       205        0
4       504        0
5       238        0
6       116        0
7       515        0
8       470        0
9       150        0
10       12        0
11      324        0
12      396        0
13       69        0
14      190        0
15      113        0
16      534        0
17      207        0
18      151        0
19      517        0
20      531        0
21       94        0
22       85        0
23      422        0
24      414        0
25      261        0
26      137        0
27      526        0
28      100        0
29      498        0
..      ...      ...
501     434        1
502     533        1
503      29        1
504     502        1
505     203        1
506      39        1
507     256        1
508     258        1
509     507        1
510     216        1
511     182        1
512     283        1
513     288        1
514      32        1
515     397        1
516     523  

Nous avons là encore un cluster plus petit que l'autre, mais d'une taille nettement plus significative.

In [45]:
acp2 = PCA(svd_solver='full')
coord2 = acp2.fit_transform(Z2)
plt.figure(figsize=(8,4))
plt.bar(np.arange(1, acp2.n_components_+1), acp2.explained_variance_ratio_*100)
plt.title("Part de la variance expliquée par chaque axe factoriel", size=16, fontweight='bold')
plt.ylabel("Variance expliquée par l'axe factoriel (en %)", size=10)
plt.xlabel("Axe factoriel", size=10)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Les deux premiers axes expliquent maintenant 50% de la variance à eux seuls.

Représentons les observations par cluster dans le premier plan factoriel. 

In [46]:
fig, axes = plt.subplots(figsize=(10,6))
axes.set_xlim(-5,10)
axes.set_ylim(-5,10)

for i in range(len(coord2[:,0])):
    if kmeans2.labels_[i] == 0:
        axes.text(coord2[i,0], coord2[i,1], str(i), size=6, color='r')
    else :
        axes.text(coord2[i,0], coord2[i,1], str(i), size=6, color='g')

axes.scatter(coord2[:,0], coord2[:,1])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

On remarque que les députés du cluster de gauche (en rouge) se distinguent essentiellement des députés du cluster vert par leur abscisse, c'est-à-dire par leur coordonnée sur leur premier axe factoriel. Nous allons donc maintenant calculer la corrélation des variables avec les deux premiers axes factoriels puis tracer le cercle des corrélations afin d'interpréter ces deux axes. 

In [47]:
n = len(df_clusters2)
p = len(df_clusters2.columns)

eigval = (n-1)/n*acp2.explained_variance_
sqrt_eigval = np.sqrt(eigval)
corvar = np.zeros((p,p))
for k in range(p):
    corvar[:,k] = acp2.components_[k,:] * sqrt_eigval[k]

print(pd.DataFrame({'Variable':df_clusters2.columns,'Cor_axe_1':corvar[:,0],'Cor_axe_2':corvar[:,1]}))

                           Variable  Cor_axe_1  Cor_axe_2
0                 semaines_presence   0.552689   0.605856
1              commission_presences   0.413833   0.690726
2          commission_interventions   0.567610   0.424653
3           hemicycle_interventions   0.817380   0.262870
4   hemicycle_interventions_courtes   0.684536   0.172765
5              amendements_proposes   0.715332  -0.193385
6                amendements_signes   0.760858  -0.365307
7               amendements_adoptes  -0.415093   0.544406
8                          rapports   0.054197   0.260701
9              propositions_ecrites   0.460106  -0.260975
10             propositions_signees   0.550433  -0.453888
11                questions_ecrites   0.473171  -0.351876
12                 questions_orales   0.769974  -0.039379


In [48]:
fig, axes = plt.subplots(figsize=(8,8))
axes.set_xlim(-1.05,1.05)
axes.set_ylim(-1.05,1.05)

for j in range(p):
    axes.arrow(0, 0, corvar[j,0], corvar[j,1], head_width=0.005, head_length=0.01, fc='grey', ec='grey')
    plt.annotate(df_clusters2.columns[j], (corvar[j,0],corvar[j,1]), size=8, color='r')

plt.plot([-1,1],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[-1,1],color='silver',linestyle='-',linewidth=1)

cercle = plt.Circle((0,0), 1, color='blue', fill=False)
axes.add_artist(cercle)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

L'immense majorité des variables d'assiduité semblent très fortement corrélées avec le premier axe factoriel (à l'exception des rapports et des amendements adoptés), alors que le deuxième axe factoriel ne semble pas vraiment rendre compte de l'assiduité des députés. On va donc s'intéresser en particulier aux députés présentant une coordonnée élevée selon le premier axe factoriel. 

On va commencer par étudier les caractéristiques des députés de chacun des deux clusters afin de dégager des tendances claires quant aux facteurs pouvant expliquer l'assiduité supérieure des députés du cluster vert par rapport à celle des députés du cluster rouge. Pour ce faire, on scinde notre table de travail en deux sous-tables contenant chacune les députés de l'un des deux clusters.

In [49]:
A = pd.DataFrame({"député" : df_clusters2.index[idk], "cluster" : kmeans2.labels_[idk]})
R = [] # Liste qui contiendra les index des députés du cluster rouge.
V = [] # Liste qui contiendra les index des députés du cluster vert.
for i in range(len(A)) :
    if A['cluster'][i] == 0 :
        R.append(A['député'][i])
    else : 
        V.append(A['député'][i])
# On vérifie qu'il y a bien 437 éléments dans la liste R et 94 dans la liste R.
len(R), len(V) 

(437, 94)

In [50]:
df_cluster_rouge = df_work.loc[R] # Sous-table avec les députés du cluster rouge.
df_cluster_vert = df_work.loc[V] # Sous-table avec les députés du cluster vert.

On va maintenant comparer les statistiques obtenues sur les variables dans les deux cas. Commençons par les variables quantitatives. 

In [51]:
df_cluster_rouge.describe()

,num_circo,nb_mandats,semaines_presence,commission_presences,commission_interventions,hemicycle_interventions,hemicycle_interventions_courtes,amendements_proposes,amendements_signes,amendements_adoptes,rapports,propositions_ecrites,propositions_signees,questions_ecrites,questions_orales,age
count,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000
mean,4.590389,1.260870,25.066362,34.926773,22.485126,39.993135,25.681922,57.013730,920.439359,162.464531,0.109840,0.105263,4.363844,13.473684,1.945080,51.846682
std,3.593342,0.460008,6.477745,17.963371,40.759058,55.352146,59.534640,95.201307,792.468492,75.542210,0.373207,0.392469,3.405725,14.442304,1.881027,11.217484
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.000000
25%,2.000000,1.000000,21.000000,22.000000,4.000000,8.000000,2.000000,12.000000,482.000000,137.000000,0.000000,0.000000,3.000000,2.000000,1.000000,44.000000
50%,3.000000,1.000000,26.000000,32.000000,12.000000,20.000000,6.000000,29.000000,646.000000,174.000000,0.000000,0.000000,4.000000,9.000000,2.000000,52.000000
75%,6.000000,2.000000,29.000000,46.000000,23.000000,47.000000,20.000000,68.000000,989.000000,211.000000,0.000000,0.000000,5.000000,19.000000,3.000000,60.000000
max,21.000000,3.000000,42.000000,102.000000,376.000000,403.000000,546.000000,873.000000,4683.000000,503.000000,3.000000,3.000000,32.000000,64.000000,12.000000,82.000000


In [52]:
df_cluster_vert.describe()

,num_circo,nb_mandats,semaines_presence,commission_presences,commission_interventions,hemicycle_interventions,hemicycle_interventions_courtes,amendements_proposes,amendements_signes,amendements_adoptes,rapports,propositions_ecrites,propositions_signees,questions_ecrites,questions_orales,age
count,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000
mean,4.563830,1.340426,31.882979,49.553191,86.255319,230.787234,219.819149,850.936170,3552.723404,90.127660,0.106383,0.808511,12.765957,29.734043,7.670213,52.808511
std,3.800279,0.539876,5.407793,20.528824,90.582575,165.031432,243.734298,957.109389,1567.971934,49.373981,0.309980,1.175709,8.345272,16.024964,3.759971,10.814443
min,1.000000,1.000000,16.000000,9.000000,3.000000,8.000000,5.000000,11.000000,514.000000,19.000000,0.000000,0.000000,1.000000,0.000000,0.000000,29.000000
25%,2.000000,1.000000,28.250000,34.000000,31.250000,114.250000,54.250000,156.500000,2517.500000,44.250000,0.000000,0.000000,7.000000,15.000000,5.000000,45.500000
50%,4.000000,1.000000,32.000000,45.000000,56.000000,175.500000,153.500000,416.500000,3465.500000,83.000000,0.000000,0.000000,11.000000,31.500000,7.000000,54.000000
75%,6.000000,2.000000,35.000000,62.250000,96.000000,320.000000,293.750000,1027.000000,5262.500000,120.750000,0.000000,1.000000,16.750000,41.750000,9.750000,61.000000
max,20.000000,3.000000,43.000000,111.000000,500.000000,708.000000,1531.000000,2887.000000,7434.000000,213.000000,1.000000,6.000000,37.000000,53.000000,18.000000,78.000000


Nous constatons que les députés du cluster vert ont bel et bien une assiduité très supérieure à leurs homologues du cluster rouge : les valeurs moyennes et médianes pour les variables de présence ou de participation sont bien plus élevées dans leur cas. La segmentation précédemment effectuée est donc pertinente. 
En revanche, l'âge ne semble pas avoir d'influence sur cette assiduité : les deux distributions semblent se superposer. Enfin, contre toute attente, cumuler des mandats n'est pas un facteur d'absentéisme ou d'inactivité ; au contraire, les députés les plus assidus ont en moyenne davantage de mandats que les autres, même si cette différence est très légère. 

Intéressons-nous désormais aux variables qualitatives, à commencer par le sexe des députés.

In [53]:
df_cluster_rouge['sexe'].value_counts() 
# Répartition des femmes et des hommes dans le cluster rouge.

H    255
F    182
Name: sexe, dtype: int64

In [54]:
df_cluster_vert['sexe'].value_counts()
# Répartition des femmes et des hommes dans le cluster vert.

H    63
F    31
Name: sexe, dtype: int64

In [55]:
(182/437)*100, (31/94)*100 # On calcule le pourcentage de femmes dans chacun des deux clusters.

(41.64759725400457, 32.97872340425532)

On remarque que les femmes sont moins représentées en pourcentage dans le cluster vert, autrement dit celui des députés assidus. Cette tendance apparaissait déjà lors de l'analyse descriptive à laquelle nous avons procédé précédemment.

Voyons désormais si les différents partis politiques sont représentés de la même manière dans les deux clusters. 

In [56]:
print(df_cluster_rouge['groupe_sigle'].value_counts()) 
print(df_cluster_vert['groupe_sigle'].value_counts())

LREM     255
LR        54
MODEM     51
NI        23
AE        17
SOC       12
LT        11
UDI       11
GDR        3
Name: groupe_sigle, dtype: int64
LR       36
LFI      17
SOC      12
GDR      12
LT        7
UDI       5
AE        2
MODEM     2
NI        1
Name: groupe_sigle, dtype: int64


On remarque ici que les députés de la majorité LREM-MODEM sont intégralement dans le cluster des députés les moins assidus, à l'exception de deux députés du MODEM. A l'inverse, les députés de gauche, en particulier du GDR (Groupe de la Gauche démocrate et républicaine) et du groupe LFI (La France insoumise), se distinguent par une présence quasi-exclusive au sein du groupe de députés assidus (ici le cluster vert).

Si l'âge ne semble pas corrélé avec l'assiduité, il apparaît que le sexe et l'appartenance politique le sont davantage : les hommes et les parlementaires de gauche sont globalement plus assidus que les femmes et les parlementaires de la majorité.